In [25]:
%pip install fuzzywuzzy
%pip install rapidfuzz
%pip install pandas
%pip install ultralytics
%pip install easyocr
%pip install python-Levenshtein
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from rapidfuzz.distance import Levenshtein
from ultralytics import YOLO
import pandas as pd
import numpy as np
import easyocr
import cv2
import glob
import json

In [18]:
your_path = 'test/imgs'

In [9]:
# Загрузка обученной модели YOLOv8
model = YOLO("best.pt")  # замените на вашу лучшую модель

# Инициализация EasyOCR
reader = easyocr.Reader(['en'])  # добавьте 'ru' для русского текста

In [4]:
def save_results_to_csv(recognized_data, output_csv="submission.csv"):
    # Преобразуем список данных в DataFrame
    df = pd.DataFrame(recognized_data, columns=["image_file", "label", "label_text"])
    # Сохраняем DataFrame в CSV файл
    df.to_csv(output_csv, index=False, encoding="utf-8-sig")

In [5]:
def filter_and_average_similar_detections(detections, threshold=0.015):
    filtered_detections = []

    for det in detections:
        det.insert(0,0)

        x_center, y_center, width, height = det[1:5]
        is_similar = False

        # Проверяем каждую детекцию на схожесть с уже добавленными в filtered_detections
        for i, fd in enumerate(filtered_detections):
            fd_x_center = fd[1]
            
            # Проверяем схожесть только по координате x_center
            if abs(x_center - fd_x_center) < threshold:
                # Если схожие детекции найдены, усредняем значения
                avg_detection = [
                    det[0],  # Класс остается тем же
                    np.mean([x_center, fd_x_center]),
                    np.mean([y_center, fd[2]]),
                    np.mean([width, fd[3]]),
                    np.mean([height, fd[4]])
                ]
                
                # Заменяем старую детекцию на усреднённую
                filtered_detections[i] = avg_detection
                is_similar = True
                break

        # Если похожей детекции не было найдено, добавляем текущую как новую
        if not is_similar:
            filtered_detections.append(det)
    
    return filtered_detections

In [26]:
def detect_and_recognize_text_in_folder(folder_path):
    # Загружаем базу данных для сравнения
    df = pd.read_csv("grounded true train.csv")
    database_numbers = df["label_text"].tolist()

    recognized_data = []

    # Обрабатываем все изображения в папке
    for image_path in glob.glob(f"{folder_path}/*.JPG"):
        # Выполняем детекцию текстовых областей с YOLOv8
        results = model(image_path)
        detections = results[0].boxes.xywhn  # YOLO формат (нормализованные координаты: x_center, y_center, width, height)

        # Преобразуем detections в список
        detections = detections.tolist()

        # Применяем фильтрацию для похожих детекций
        filtered_detections = filter_and_average_similar_detections(detections)

        image = cv2.imread(image_path)
        img_h, img_w = image.shape[:2]

        detected_texts = []
        labels = []

        # Обработка каждого обнаруженного текстового блока
        for box in filtered_detections:
            x_center, y_center, width, height = box[1:5]
            
            # Преобразование координат из нормализованных в пиксельные
            x_center_pix = int(x_center * img_w)
            y_center_pix = int(y_center * img_h)
            width_pix = int(width * img_w)
            height_pix = int(height * img_h)

            x1 = int((x_center - width / 2) * img_w)
            y1 = int((y_center - height / 2) * img_h)
            x2 = int((x_center + width / 2) * img_w)
            y2 = int((y_center + height / 2) * img_h)

            cropped_image = image[y1:y2, x1:x2]  # Вырезаем область с текстом

            # Применяем OCR к обрезанному изображению
            text = reader.readtext(cropped_image, detail=0)
            detected_texts.extend(text)  # Добавляем результаты OCR

            # Сохраняем координаты рамки в формате YOLO (по пикселям)
            labels.append([0, x_center_pix / img_w, y_center_pix / img_h, width_pix / img_w, height_pix / img_h])

            # Рисуем рамку на изображении для визуализации
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Объединяем распознанный текст в одну строку
        detected_number = ' '.join(detected_texts)

        # Находим лучшее совпадение с базой данных
        best_partial_match = process.extractOne(detected_number, database_numbers, scorer=fuzz.partial_ratio)
        mark = best_partial_match[0]

        labels = '\n'.join(''.join(map(str, row)) for row in labels)
        # Сохраняем результат в список
        recognized_data.append({
            "image_file": image_path.split('/')[-1],
            "label": labels,
            "label_text": mark.strip('"')  # Удаляем лишние кавычки, если они есть
        })

        # Сохраняем изображение с нарисованными рамками
        cv2.imwrite(f"output/{image_path.split('/')[-1]}", image)

    # Сохраняем результаты в JSON файл
    save_results_to_csv(recognized_data)

    return recognized_data

In [27]:
# Запускаем обработку папки
results = detect_and_recognize_text_in_folder(f"{your_path}")


image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new2.JPG: 480x640 1 text, 343.3ms
Speed: 5.0ms preprocess, 343.3ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new137.JPG: 480x640 1 text, 302.1ms
Speed: 2.1ms preprocess, 302.1ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new123.JPG: 640x480 1 text, 216.4ms
Speed: 1.8ms preprocess, 216.4ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new86.JPG: 640x480 1 text, 305.3ms
Speed: 2.2ms preprocess, 305.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new92.JPG: 480x640 1 text, 269.1ms
Speed: 2.3ms preprocess, 269.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new79.JPG: 640x480 2 t


image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new95.JPG: 640x480 1 text, 202.3ms
Speed: 2.3ms preprocess, 202.3ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new130.JPG: 480x640 1 text, 204.0ms
Speed: 2.4ms preprocess, 204.0ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new124.JPG: 640x480 1 text, 213.3ms
Speed: 2.0ms preprocess, 213.3ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new126.JPG: 480x640 1 text, 206.3ms
Speed: 3.2ms preprocess, 206.3ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new97.JPG: 640x480 1 text, 194.4ms
Speed: 1.6ms preprocess, 194.4ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new68.JPG: 640x480 1


image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new141.JPG: 640x480 1 text, 376.0ms
Speed: 3.2ms preprocess, 376.0ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new143.JPG: 640x480 1 text, 274.2ms
Speed: 2.9ms preprocess, 274.2ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new31.JPG: 480x640 2 texts, 321.9ms
Speed: 3.4ms preprocess, 321.9ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new24.JPG: 640x512 1 text, 247.2ms
Speed: 3.7ms preprocess, 247.2ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 512)

image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new30.JPG: 480x640 1 text, 646.5ms
Speed: 3.1ms preprocess, 646.5ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new156.JPG: 480x640 


image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new164.JPG: 480x640 1 text, 216.1ms
Speed: 3.4ms preprocess, 216.1ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new158.JPG: 480x640 2 texts, 223.5ms
Speed: 2.5ms preprocess, 223.5ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new17.JPG: 640x480 1 text, 210.9ms
Speed: 1.7ms preprocess, 210.9ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new159.JPG: 480x640 1 text, 239.2ms
Speed: 3.2ms preprocess, 239.2ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new116.JPG: 640x480 1 text, 190.2ms
Speed: 1.8ms preprocess, 190.2ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new102.JPG: 640x48


image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new98.JPG: 640x480 1 text, 219.8ms
Speed: 2.1ms preprocess, 219.8ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new99.JPG: 640x480 1 text, 245.7ms
Speed: 3.5ms preprocess, 245.7ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new114.JPG: 640x480 1 text, 213.3ms
Speed: 2.1ms preprocess, 213.3ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new100.JPG: 640x480 1 text, 213.9ms
Speed: 2.0ms preprocess, 213.9ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new128.JPG: 640x480 1 text, 201.1ms
Speed: 1.6ms preprocess, 201.1ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new110.JPG: 640x480 


image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new106.JPG: 640x480 1 text, 223.1ms
Speed: 2.0ms preprocess, 223.1ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /Users/andrew/Desktop/TEXTHACK/test/imgs/new112.JPG: 640x480 1 text, 203.6ms
Speed: 1.7ms preprocess, 203.6ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 480)
